Theoretical questions
Q1: To evaluate how well our model performs at T1D classification, we need to have evaluation metrics that measures of its performances/accuracy. Which evaluation metric is more important to us: model accuracy or model performance? Give a simple example that illustrates your claim.

A1: Performance means how good our model is doing its job. Accuracy is the number of correct predictions made by the model by the total number of records. For us, performance is more important beacuse accuracy can be deceiving if the data is not representive enough.
For instance, if we use a binary naive classifier to determine whether a person has cancer, However the data only includes 5 positive cases out of 100 patients, we get 95% accuracy.

Q2: T1D is often associated with other comorbidities such as a heart attack. You are asked to design a ML algorithm to predict which patients are going to suffer a heart attack. Relevant patient features for the algorithm may include blood pressure (BP), body-mass index (BMI), age (A), level of physical activity (P), and income (I). You should choose between two classifiers: the first uses only BP and BMI features and the other one uses all of the features available to you. Explain the pros and cons of each choice.

A2: 

Using BMI and BP:
    
    pros
      - Weights will be easier to find.
      - The calculations will be simpler.
      - Data cleanning will only include two coloumns.
    cons
      - We can't be sure that these are the most important features. This can lead to diminished performance and crucial mistakes. 
        For example, a false negative case; One may not be treated well which can lead to devastating consequences.
    
Using all features:

    pros
      - The model will most likely perform better and will have better accuracy (under the assumption that our data is reliable enough)
    cons
      - Data exploration and cleanning will be difficult.
      - We'll have a lot of data meanning extended computation time.
    
        

Q3: A histologist wants to use machine learning to tell the difference between pancreas biopsies that show signs of T1D and those that do not. She has already come up with dozens of measurements to take, such as color, size, uniformity and cell-count, but she isn’t sure which model to use. The biopsies are really similar, and it is difficult to distinguish them from the human eye, or by just looking at the features. Which of the following is better: logistic regression, linear SVM or nonlinear SVM? Explain your answer.

A3:

We can assume that the data won't be linearly seperable since the samples look alike and have similar properties.
Thus, we will use nonlinear SVM to find the correct boundary line.

Q4: What are the differences between LR and linear SVM and what is the difference in the effect/concept of their hyper-parameters tuning?

A4: LR - returns the probabilities to be a part of a certain class. 

Linear SVM - returns where each patient will be labeled.

There is one hyperparameter to tune - lambda (or C in python which is 1/lambda). 
This hyperparameter determines the bias - variance tradeoff, meaning that for low values of lambda (or high values of C), our model can be complex and we risk overfitting our training examples(high variance). in this case the model will memorise, not learn.

For high values of lambda (or low values of C) we "punish" harder our model for each misclassification and risk having an underfit condition, meaning that our model will be too simple (high bias).


Next, we import the relevant packages and write two functions.
The first is nan2num, which replaces all nan values with random numbers from the same coloumn.

The second is str_to_bool_series which will help us classify later.
Except age, all categories can be binarized as follows:

Yes = true ; No = false

Male = true ; Female = false

Positive = True ; Negative = false

In [1]:
import pandas as pd 
import numpy as np
from pathlib import Path
import random
import distutils
from sklearn import metrics
from sklearn.model_selection import train_test_split
df = pd.read_csv('HW2_data.csv')

In [2]:
def nan2num(dataframe):
    
    #:param dataframe: Pandas series of features
    #:return: A pandas dataframe of the dictionary c_cdf containing the "clean" features
    
    # dataframe = pd.DataFrame(dataframe).drop(extra_feature, 1)
    c_cdf = {}
    c_cdf = dataframe
    for column in dataframe.columns:
        if column == 'Age':
            i=0
        else:
            replacing_value = np.random.choice(dataframe[column])
            c_cdf[column].replace(to_replace=np.nan, value=replacing_value, inplace=True)
    return pd.DataFrame(c_cdf)

def str_to_bool_series(s):
    for index, value in s.items():
        if(value=='Yes'):
            s[index] = True
        elif(value=='No'):
            s[index] = False
        elif(value=='Male'):
            s[index] = True
        elif(value == 'Female'):
            s[index] = False
        elif(value=='Positive'):
            s[index] = True
        elif (value == 'Negative'):
            s[index] = False
#         elif(value==1):
#             s[index] = True
#         elif(value==0):
#             s[index] = False
    return s

Now, we get the clean data:

In [4]:
df=nan2num(df)
for coloumn in df.columns:
    df[coloumn]=str_to_bool_series(df[coloumn])
print(df)

     Age  Gender  Increased Urination  Increased Thirst  Sudden Weight Loss  \
0     45    True                False             False               False   
1     42    True                False             False               False   
2     45    True                 True              True               False   
3     59   False                False             False               False   
4     40   False                 True              True                True   
..   ...     ...                  ...               ...                 ...   
560   54    True                 True              True                True   
561   32    True                False             False               False   
562   61   False                 True             False               False   
563   46    True                False             False               False   
564   37    True                False             False               False   

     Weakness  Increased Hunger  Genital Thrush  Vi

Train - Test split:

In [5]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=0)